In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [21]:
print("Hello, We are going to start Music generation in kaggle environment")

Hello, We are going to start Music generation in kaggle environment


In [22]:
!mkdir dataset

mkdir: cannot create directory ‘dataset’: File exists


/usr/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [23]:
import os
import music21 as m21
import json
import tensorflow.keras as keras
import numpy as np

# Settings for environment setup for MuseScore
# env = m21.environment.Environment()
# env['musicxmlPath'] = r"D:\Program Files\MuseScore 4\bin\MuseScore4.exe"
# env['musescoreDirectPNGPath'] = r"D:\Program Files\MuseScore 4\bin\MuseScore4.exe"

# Standard variables
DATASET_PATH = '/kaggle/input/music-generation/deutschl/erk'
ACCEPTED_DURATION = [0.25, 0.5, 0.75, 1.0, 1.5, 2, 3, 4]
SAVE_DIR = '/kaggle/working/dataset'
SINGLE_FILE_DATASET = '/kaggle/working/file_dataset'
SEQUENCE_LENGTH = 64
MAPPING_PATH = '/kaggle/working/mapping.json'


In [24]:

# Loading the song
def load_song(dataset_path):
    songs = []
    # Go through all the files in the dataset and load them with music21
    for path, subdirs, files in os.walk(dataset_path):
        for file in files:
            if file.endswith(".krn"):
                song = m21.converter.parse(os.path.join(path, file))
                songs.append(song)
                
    return songs

# Filtering out the songs that have non-acceptable duration
def has_acceptable_durations(song, accepted_durations):
    for note in song.flatten().notesAndRests:
        if note.duration.quarterLength not in accepted_durations:
            return False
    return True


def transpose(song):
    
    # get key from the song
    parts = song.getElementsByClass(m21.stream.Part)
    measures_part0 = parts[0].getElementsByClass(m21.stream.Measure)
    key = measures_part0[0][4]
    
    # Estimate key using music21
    if not isinstance(key, m21.key.Key):
        key = song.analyze('key')
    
    # get interval from transposition E.g., Bmaj -> Cmaj
    if key.mode == 'major':
        interval = m21.interval.Interval(key.tonic, m21.pitch.Pitch('C'))
    elif key.mode == 'minor':
        interval = m21.interval.Interval(key.tonic, m21.pitch.Pitch('A'))
    
    # Transpose song by calculated interval
    transposed_song = song.transpose(interval)
    
    return transposed_song
    

def encode_song(song, time_step=0.25):
    
    # p=60, d=1.0 -> [60, "_", "_", "_"]
    # So we need to encode the notes and rests
    
    encoded_song = []
    
    for event in song.flatten().notesAndRests:
    
        # handling notes
        if isinstance(event, m21.note.Note):
            symbol = event.pitch.midi # 60
        # handling rests
        elif isinstance(event, m21.note.Rest):
            symbol = "r"
        
        # Convert the note/rest into time series representation
        steps = int(event.duration.quarterLength / time_step)
        for step in range(steps):
            if step == 0:
                encoded_song.append(symbol)
            else:
                encoded_song.append("_")
        
    # Cast the encoded song to a string
    encoded_song = " ".join(map(str, encoded_song))
    
    return encoded_song



# Preprocessing the data
def preprocess_data(dataset_path):
    
    '''Load the folk songs'''
    print("\nLoading songs...")
    songs = load_song(dataset_path)
    print(f"Loaded {len(songs)} songs.")
    
    # For MuseScore graph
    # song = songs[0]
    # song.show()
    
    print("Starting preprocessing....")
    for i, song in enumerate(songs):    
        
        '''Filter out the songs that have non-acceptable duration'''
        if not has_acceptable_durations(song, ACCEPTED_DURATION):
            # print(f"Song has non-acceptable duration, skipping it...")
            continue

        '''Transpose song to Cmaj/Amin'''
        # print("Starting the transposition...")
        song = transpose(song)
        
        '''Encode songs with music time series representation'''
        # print("Encoding the song...")
        encoded_song = encode_song(song)
        
        '''Save song to text file'''
        # print("Saving the song...")
        save_path = os.path.join(SAVE_DIR, str(i))
        with open(save_path, "w+") as file:
            file.write(encoded_song)
            
        # print(f"Song {i} saved at {save_path}")
    
    print("\nPreprocessing finished.")

def load(file_path):
    with open(file_path, "r") as file:
        song = file.read()
    return song


def create_single_file_datasets(dataset_path, file_dataset_path, sequence_length):
    
    print("starting create_single_file_datasets")
    
    new_song_delimiter = "/ "*sequence_length
    
    songs = ""
    
    '''Load encoded songs and delimiter'''
    for path, _, files in os.walk(dataset_path):
        for file in files:
            file_path = os.path.join(path, file)
            song = load(file_path)
            songs += song + " " + new_song_delimiter
    
    songs = songs[:-1]
    
    '''Save sttring that contain all the dataset'''
    with open(file_dataset_path, "w") as file:
        file.write(songs)
        
    print("Done with create_single_file_datasets")

    return songs

def create_mapping(songs, mapping_path):
    
    print("Starting create_mapping")
    
    mappings = {}
    
    '''Identify the vocabulary'''
    songs = songs.split()
    vocabulary = list(set(songs))
    
    '''Create the mappings'''
    for i, symbol in enumerate(vocabulary):
        mappings[symbol] = i
    
    '''Save the vocab to json file'''
    with open(mapping_path, "w") as file:
        json.dump(mappings, file, indent=4)
    
    print("Done with create_mapping")
    
    
def convert_songs_to_int(songs):
    
    int_songs = []
    
    '''Load mappings'''
    with open(MAPPING_PATH, "r") as file:
        mappings = json.load(file)
    
    '''cast songs string to a list'''
    songs = songs.split()
    
    '''map song to int'''
    for symbol in songs:
        int_songs.append(mappings[symbol])
    
    return int_songs
    
def generate_training_sequences(sequence_length):
    print("Starting generate_training_sequences")
    # [11, 12, 13, 14 ...] -> input: [11, 12], target: [13], i:[11, 12, 13], t:[14]
    
    '''load the songs and map them to int'''
    songs = load(SINGLE_FILE_DATASET)
    int_songs = convert_songs_to_int(songs)
        
    '''generate the training sequences'''
    # how many seq should/can we generate
    # 100 symbols, 64 seq len, 100-36 no of seq
    
    inputs = []
    targets = []
    
    num_sequences = len(int_songs) - sequence_length
    for i in range(num_sequences):
        inputs.append(int_songs[i:i+sequence_length])
        targets.append(int_songs[i+sequence_length])
    
    '''One hot encode the sequences'''
    # input: (no of seq, seq len)
    vocabulary_size = len(set(int_songs))
    inputs = keras.utils.to_categorical(inputs, num_classes=vocabulary_size)
    targets = np.array(targets)
    
    print("dimension of input and target are: ", inputs.shape, targets.shape)
    return inputs, targets
   

In [4]:
 
def main():
    preprocess_data(DATASET_PATH)
    songs = create_single_file_datasets(SAVE_DIR, SINGLE_FILE_DATASET, SEQUENCE_LENGTH)
    create_mapping(songs, MAPPING_PATH)
    inputs, targets = generate_training_sequences(SEQUENCE_LENGTH)
    


if __name__ == "__main__":    
    main()


Loading songs...
Loaded 1700 songs.
Starting preprocessing....

Preprocessing finished.
starting create_single_file_datasets
Done with create_single_file_datasets
Starting create_mapping
Done with create_mapping
Starting generate_training_sequences
dimension of input and target are:  (362178, 64, 38) (362178,)


In [25]:
import tensorflow.keras as keras
import sys
# import io
# sys.stdout = io.TextIOWrapper(sys.stdout.buffer, encoding='utf-8')



OUTPUT_UNITS = 38
LOSS = "sparse_categorical_crossentropy"
LEARNING_RATE = 0.001
NUM_UNITS = [256]
EPOCHS = 50
BATCH_SIZE = 64
SAVE_MODEL_PATH = "/kaggle/working/model.h5"

In [26]:
# Generate the training sequences
print("On step 1")
train_inputs, targets = generate_training_sequences(SEQUENCE_LENGTH)


On step 1
Starting generate_training_sequences
dimension of input and target are:  (362178, 64, 38) (362178,)


In [7]:
# build the network
output_units = OUTPUT_UNITS
loss = LOSS
learning_rate = LEARNING_RATE
num_units = NUM_UNITS
epochs = EPOCHS
batch_size = BATCH_SIZE

print("On step 2")
# Create model architecture
build_inputs = keras.layers.Input(shape=(None, output_units))
x = keras.layers.LSTM(num_units[0])(build_inputs)
x = keras.layers.Dropout(0.2)(x)
build_outputs = keras.layers.Dense(output_units, activation='softmax')(x)
model = keras.Model(build_inputs, build_outputs)

# Compile the model
model.compile(
loss=loss,
optimizer = keras.optimizers.Adam(learning_rate=learning_rate),
metrics = ['accuracy']
)

model.summary()

On step 2


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, None, 38)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (None, 256)                 │         302,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 38)                  │           9,766 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 311,846 (1.19 MB)

 Trainable params: 311,846 (1.19 MB)

 Non-trainable params: 0 (0.00 B)

In [8]:


# train the model
print("On step 3")
model.fit(train_inputs, targets, epochs=EPOCHS, batch_size=BATCH_SIZE)


On step 3
Epoch 1/50
5660/5660 ━━━━━━━━━━━━━━━━━━━━ 39s 6ms/step - accuracy: 0.7681 - loss: 0.8364
Epoch 2/50
5660/5660 ━━━━━━━━━━━━━━━━━━━━ 36s 6ms/step - accuracy: 0.8056 - loss: 0.6050
Epoch 3/50
5660/5660 ━━━━━━━━━━━━━━━━━━━━ 37s 7ms/step - accuracy: 0.8190 - loss: 0.5613
Epoch 4/50
5660/5660 ━━━━━━━━━━━━━━━━━━━━ 38s 7ms/step - accuracy: 0.8283 - loss: 0.5315
Epoch 5/50
5660/5660 ━━━━━━━━━━━━━━━━━━━━ 37s 7ms/step - accuracy: 0.8341 - loss: 0.5117
Epoch 6/50
5660/5660 ━━━━━━━━━━━━━━━━━━━━ 38s 7ms/step - accuracy: 0.8389 - loss: 0.4944
Epoch 7/50
5660/5660 ━━━━━━━━━━━━━━━━━━━━ 38s 7ms/step - accuracy: 0.8466 - loss: 0.4701
Epoch 8/50
5660/5660 ━━━━━━━━━━━━━━━━━━━━ 38s 7ms/step - accuracy: 0.8495 - loss: 0.4582
Epoch 9/50
5660/5660 ━━━━━━━━━━━━━━━━━━━━ 38s 7ms/step - accuracy: 0.8539 - loss: 0.4417
Epoch 10/50
5660/5660 ━━━━━━━━━━━━━━━━━━━━ 38s 7ms/step - accuracy: 0.8584 - loss: 0.4293
Epoch 11/50
5660/5660 ━━━━━━━━━━━━━━━━━━━━ 38s 7ms/step - accuracy: 0.8628 - loss: 0.4137
Epoch 12/

In [9]:

# Save the model
print("On step 4")
model.save(SAVE_MODEL_PATH)



On step 4


## Melody generator

In [27]:
import tensorflow.keras as keras
import json
import numpy as np
import music21 as m21

In [30]:
class MelodyGenerator:

    def __init__(self ,model_path = SAVE_MODEL_PATH):
        
        self.model_path = model_path
        self.model = keras.models.load_model(model_path)

        with open(MAPPING_PATH, "r") as file:
            self._mappings = json.load(file)

        self._start_symbols = ["/"] * SEQUENCE_LENGTH

    def generate_melody(self, seed, num_steps, max_sequence_length, temperature):
        '''
        seed is a peice of melody
        "64 _ 63 _ _ ...."
        '''
        
        '''Create seed with start symbol'''
        seed = seed.split()
        melody = seed
        seed = self._start_symbols + seed

        '''map seed to integers'''
        seed = [self._mappings[symbol] for symbol in seed]

        for _ in range(num_steps):
            '''limit the seed to max_seqence_length'''
            seed = seed[-max_sequence_length:]

            '''one hot encode the seed'''
            onehot_seed = keras.utils.to_categorical(seed, num_classes=len(self._mappings))
            # shape -> (max_sequence_length, num_symbols)
            # convert into (1, max_sequence_length, num_symbols)
            onehot_seed = onehot_seed[np.newaxis, ...]

            '''Make a prediction'''
            probabilities = self.model.predict(onehot_seed)[0]
            # [0.1, 0.2, 0.1, 0.6] -> sum -> 1

            '''We will use temperature sampling'''
            output_int = self._sample_with_temperature(probabilities, temperature)

            '''Update the seed'''
            seed.append(output_int)

            '''Map int to our encoding'''
            output_symbol = [k for k,v in self._mappings.items() if v == output_int][0]
            
            '''Check whether we are at the end of the melody'''
            if output_symbol == "/":
                break

            '''update the melody'''
            melody.append(output_symbol)
        return melody
            

    
    def _sample_with_temperature(self ,probabilities, temperature):
        '''
        Here we want an index, we won't use np.argmax directly as thats rigid
        We want something more flexible

        temperature -> infinity -> This will lead to randommness which is not good
        temperature -> 0 -> This lead to same as argmax which chooses with max prob
        temperature ->1 1 -> Normal dist, we return the same
        
        '''   
        predictions = np.log(probabilities) / temperature
        # Now apply softmax
        probabilities = np.exp(predictions) / np.sum(np.exp(predictions))
        # Now we get more homogenous distribution

        '''Sampling index'''
        choices = range(len(probabilities))
        index = np.random.choice(choices, p=probabilities)

        return index

    def save_melody(self, melody, step_duration=0.25, format="midi", file_name = "/kaggle/working/mel.midi"):
        
        '''Create a music21 stream'''
        stream = m21.stream.Stream()
        
        '''parse all the symbol in the melody and create note/rest objects'''
        # 60 _ _ _ r _ 62 _
        start_symbol = None
        step_counter = 1
        
        for i, symbol in enumerate(melody):

            # Handle case in which we have a note/rest
            if symbol != "_" or i+1 == len(melody):
                # Ensure we are dealing with note/rest beyinf first one
                if start_symbol is not None:

                    # calculate the quarter length duration
                    quarter_length_duration = step_duration*step_counter
                    
                    # handle rest
                    if start_symbol == 'r':
                        m21_event = m21.note.Rest(quarterLength=quarter_length_duration)

                    # handle note
                    else:
                        m21_event = m21.note.Note(int(start_symbol), quarterLength=quarter_length_duration)

                    stream.append(m21_event)

                    # Reset the step counter
                    step_counter = 1

                start_symbol = symbol

            # Handle case in which we have prolongation sign "_"
            else:
                step_counter += 1

        '''write the m21 string to midi file'''
        stream.write(format, file_name)

        

In [ ]:
with open()

In [32]:
mg = MelodyGenerator()
seed = "67 _ _ _ _ _ 65 _ 64 _ 62 _ 60 _ _ _"
melody = mg.generate_melody(seed, 500, SEQUENCE_LENGTH, 0.4)
print(melody)
mg.save_melody(melody, step_duration=0.25, format="midi", file_name = "/kaggle/working/mel2.midi")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━

<ipython-input-30-274a6a3c091e>:70: RuntimeWarning: divide by zero encountered in log
  predictions = np.log(probabilities) / temperature


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━